In [1]:
import Pkg;
Pkg.add("JuMP") 
Pkg.add("HiGHS")

    Updating registry at `C:\Users\dieme\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\dieme\.julia\environments\v1.8\Manifest.toml`


In [2]:
########################################################################
# ht_model_v1.0.0.jl
#
# Coordenação hidro-termo modular em Julia (versão V1.0.0)
# Usa JuMP + HiGHS, com produção hidráulica PWL4
# Modularização em componentes: hidráulico, térmico e cenário
########################################################################

using JuMP
using HiGHS

# -------------------------
# 1) Definições de parâmetros e leituras de dados
# -------------------------
struct HydroPlant
    V0::Float64
    Vmin::Float64
    Vmax::Float64
    Qmax::Float64
    pwl_c::Vector{Float64}
    pwl_d::Vector{Float64}
end

struct ThermalPlant
    Pmin::Float64
    Pmax::Float64
    cost::Float64
end

struct Scenario
    T::Int
    D::Vector{Float64}
    Inflow::Dict{Tuple{Int,Int},Float64}
end

# Função para carregar parâmetros de exemplo ou de arquivo
function load_parameters()
    # Exemplo fixo: substituir por leitura de CSV, DB, etc.
    NH, NT, T = 3, 2, 12
    # Usinas hidráulicas
    hydros = [
        HydroPlant(50.0, 20.0, 100.0, 200.0, [0.008,0.006,0.004], [0.0,5.0,10.0]),
        HydroPlant(30.0, 10.0, 80.0, 150.0, [0.007,0.005,0.003], [0.0,4.0,8.0]),
        HydroPlant(45.0, 15.0, 90.0, 180.0, [0.009,0.007,0.005], [0.0,6.0,12.0])
    ]
    # Usinas térmicas
    thermals = [ThermalPlant(0.0,200.0,150.0), ThermalPlant(0.0,150.0,180.0)]
    # Cenário (demanda e inflows)
    D = [250.0,260.0,240.0,255.0,270.0,265.0,250.0,260.0,245.0,255.0,260.0,270.0]
    Inflow = Dict{Tuple{Int,Int},Float64}()
    for i in 1:NH, t in 1:T
        Inflow[(i,t)] = 5.0 + 2.0*rand()
    end
    Scenario(T, D, Inflow), hydros, thermals
end

# -------------------------
# 2) Montagem do modelo JuMP
# -------------------------
function build_model(scenario::Scenario, hydros::Vector{HydroPlant}, thermals::Vector{ThermalPlant})
    model = Model(HiGHS.Optimizer)
    NH, NT, T = length(hydros), length(thermals), scenario.T

    # Variáveis
    @variables(model, begin
        # Hidro
        0 <= Q[i=1:NH, t=1:T] <= hydros[i].Qmax
        0 <= S[i=1:NH, t=1:T]
        V[i=1:NH, t=1:T]
        0 <= P_H[i=1:NH, t=1:T]
        # Térmica
        P_T[j=1:NT, t=1:T] >= 0
    end)

    # Restrições Hidro
    for i in 1:NH
        # balanço hídrico
        @constraint(model, V[i,1] == hydros[i].V0 + scenario.Inflow[(i,1)] - Q[i,1] - S[i,1])
        for t in 2:T
            @constraint(model, V[i,t] == V[i,t-1] + scenario.Inflow[(i,t)] - Q[i,t] - S[i,t])
        end
        # limites de volume
        @constraint(model, [t=1:T], hydros[i].Vmin <= V[i,t] <= hydros[i].Vmax)
        # PWL4: produção hidráulica concava
        Hi = length(hydros[i].pwl_c)
        @constraint(model, [t=1:T, h=1:Hi], P_H[i,t] <= hydros[i].pwl_c[h]*Q[i,t] + hydros[i].pwl_d[h])
    end

    # Restrições Térmicas
    @constraint(model, [j=1:NT, t=1:T],
        thermals[j].Pmin <= P_T[j,t] <= thermals[j].Pmax
    )

    # Atendimento à demanda
    @constraint(model, [t=1:T],
        sum(P_H[i,t] for i in 1:NH) + sum(P_T[j,t] for j in 1:NT) == scenario.D[t]
    )

    # Objetivo de custo térmico
    @objective(model, Min, sum(thermals[j].cost * P_T[j,t] for j=1:NT, t=1:T))

    return model, Q, S, V, P_H, P_T
end

# -------------------------
# 3) Execução e extração de resultados
# -------------------------
function run()
    scenario, hydros, thermals = load_parameters()
    model, Q, S, V, P_H, P_T = build_model(scenario, hydros, thermals)
    optimize!(model)
    if termination_status(model) == MOI.OPTIMAL
        println("Status: Ótimo encontrado")
        println("Custo total (Reais): ", objective_value(model))
        println()
        # Mostrar geração térmica para cada usina e período
        println("=== Geração Térmica (P_T) ===")
        for j in 1:length(thermals), t in 1:scenario.T
            println("P_T(", j, ",", t, ") = ", value(P_T[j,t]))
        end
        println()
        # Mostrar geração hidráulica para cada usina e período
        println("=== Geração Hidráulica (P_H) ===")
        for i in 1:length(hydros), t in 1:scenario.T
            println("P_H(", i, ",", t, ") = ", value(P_H[i,t]))
        end
        println()
        # Mostrar volume armazenado para cada usina e período
        println("=== Volume Armazenado (V) ===")
        for i in 1:length(hydros), t in 1:scenario.T
            println("V(", i, ",", t, ") = ", value(V[i,t]))
        end
    else
        println("Otimização não atingiu solução ótima. Status = ",
                termination_status(model))
    end
end

# Executar a função run() em script ou notebook
run()



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
84 rows, 165 cols, 270 nonzeros
48 rows, 129 cols, 198 nonzeros
48 rows, 93 cols, 162 nonzeros
Presolve : Reductions: rows 48(-168); columns 93(-75); elements 162(-315)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.0681325277e-05 Pr: 12(3080) 0s
         72     4.8197593730e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 72
Objective value     :  4.8197593730e+05
HiGHS run time      :          0.01
Status: Ótimo encontrado
Custo total (Reais): 481975.9372993914

=== Geração Térmica (P_T) ===
P_T(1,1) = 200.0
P_T(1,2) = 200.0
P_T(1,3) = 200.0
P_T(1,4) = 200.0
P_T(1,5) = 200.0
P_T(1,6) = 200.0
P_T(1,7) = 200.0
P_T(1,8) = 200.0
P_T(1,9) = 200.0
P_T(1,10) = 200.0
P_T(1,11) = 200.0
P_T(1,12) = 200.0
P_T(2,1) = 50.0
P_T(2,2) = 60.0
P_T(2